# Syntax of PartiQL

The syntax of PartiQL is completely provisional, but there are reasons behind each choice. In this notebook, I will explain those reasons.

## Why not SQL?

Although I want to demonstrate the value of relational concepts like surrogate indexes and set operations for particle physics, strict SQL would limit usability in ways that would be too distracting for the demo.

   * PartiQL indexes are not visible, but SQL's are. In PartiQL, we are only using the indexes and index-matching to ensure that particles retain their identity, so there is only one choice for the `ON` clause of a `JOIN`. SQL is more general: SQL users sometimes want to match on surrogate keys, sometimes natural keys, and their choice will depend on their domain. Following this path, PartiQL should at least drop SQL's `ON` clause.
   * SQL's {database, table, row} hierarchy corresponds to the awkward array structure `ListArray(RecordArray(PrimitiveArrays...)))`. In particle physics, we want to deal with more structures than this. It would be possible in SQL by emulating deeper structures using table normalization, but that would require an `ON` clause to select the right foreign keys to link tables. Arguably, what PartiQL does is internally manage foreign keys with its implicit `ON` clauses to provide the appearance of deeply nested data structures, which makes it more high-level than SQL: it maintains data in a way that is appropriate for particle physics only.
   * If we apply queries to individual events, we will need new constructs to perform operations across events, such as cutting events and histogramming.
   * SQL seems to have bad design decisions, patched over by decades of practice (e.g. [common table expressions](https://www.citusdata.com/blog/2018/08/09/fun-with-sql-common-table-expressions/) is a heavy boilerplate and out-of-order way to do functional composition, the evaluation order is visible and [very different](https://sqlbolt.com/lesson/select_queries_order_of_execution) from the order in which queries are written, etc.). Adhering to SQL's syntax would put a burden on physicists who are new to it.
   * PartiQL queries are never going to be exact SQL queries, so the value of interoperability is at the level of concepts: for that, we use the same names. Data scientists who know SQL will find familiar ideas in PartiQL and physicists who learn something like PartiQL will recognize those terms when they encounter them in SQL.
   * The `cut/vary/hist` block syntax of my October 2018 language was a good idea and should be replicated here.

## General design

PartiQL has an expression syntax like Python's, including Python's operator precedence. It has the standard binary operators `+`, `-`, `*`, `/`, and `**`, comparison operators (not chained as in Python, but that would be a good idea), and `==` checks for equality. The logical operators are words: `and`, `or`, `not`, and there are no bitwise operators. Use `in` or `not in` to determine whether a value is a member of a set.

Python and C/C++ comments are both supported.

As we will see in the notebook about runtime evaluation, missing data are supported but not explicitly visible to the user as a `None` or `null` object. (A set containing missing data is the same as an empty set and a missing value for a record field is the same as not having that field.) To support missing data handling, PartiQL has [safe navigation operators](https://en.wikipedia.org/wiki/Safe_navigation_operator) `?possibly_missing` and `not_missing?.possibly_missing`, as well as a `has possibly_missing` expression that returns true or false.

Rather than being a statement, `if` is a ternary expression: `x = if a > 0 then b else c`. Without the `else` clause, it may return missing data.

Join operations are words, `join`, `cross`, `union`, `except`, and are infix binary operators, just as they are in SQL. However, they form part of the general expression syntax with other binary operators (e.g. `+`, `-`, `*`, `/`), though with lower precedence. Similarly, `where`, `group by` and `min by`/`max by` are infix binary operators.

The `X as x` syntax promotes any set to a set of records with field `x` containing the original data. This can prevent set operations from overlapping data when joining sets of records with the same field names. When multiple variable names are provided, `X as (x1, x2)` (parentheses are required), it samples without replacement, which acts as a counterpoint to `X as x1 cross X as x2` to sample with replacement.

Curly brackets are used (1) to isolate scope, so that temporary variables do not overshadow other variables with the same name, (2) to add data to a set of records after a `with` keyword, and (3) to enclose nested `cut/vary/hist` statements.

Whitespace is not significant, except that a newline can end a statement (as can a semicolon `;`). Nesting is expressed with curly brackets `{...}`, not indentation.

The outermost structures can be `cut` (to select events) and/or `vary` (to compute a block with different inputs for systematic variations). These statements are like the `region` and `vary` of the October 2018 language, but not as extensive and interchangeable with histogram binning specifiers. (It would be good to make them so.) Like the October 2018 language, `cut` and `vary` can be nested within each other, and doing so builds a hierarchy of counters for cut flows. Events have weights, which are multiplied by expressions in `weight by ...` clauses as an event descends the hierarchy.

Histograms can be placed anywhere in the hierarchy with a `hist` statement, and the corresponding histogram appears in the hierarchy of counters. This way, the binning, name, and title of the histogram are specified next to the expressions to fill and `weight by`. The number of entries in a histogram depends on its location in the hierarchy of cuts and whether it is in any `with` expressions for a set of records.

Although the language has no functions, macros (defined with `def`) are expanded during parsing to avoid repetitive typing. Macros are not runtime objects—it is as though all functions are inlined.

## Illustrative examples

To execute the examples, be sure to have the [Lark parser](https://github.com/lark-parser/lark#readme) installed.

In [1]:
!pip install lark-parser

In [2]:
import parser

Simple expressions look like any modern language.

### Scalar expressions

In [3]:
parser.parse(r"(-b + sqrt(b**2 -4*a*c))/(2*a)")

[Call(Symbol('/'), [Call(Symbol('+'), [Call(Symbol('*-1'), [Symbol('b')]), Call(Symbol('sqrt'), [Call(Symbol('-'), [Call(Symbol('**'), [Symbol('b'), Literal(2)]), Call(Symbol('*'), [Call(Symbol('*'), [Literal(4), Symbol('a')]), Symbol('c')])])])]), Call(Symbol('*'), [Literal(2), Symbol('a')])])]

Logical operations are words (like Python) and the precedence order is respected (e.g. `and` before `or`).

In [4]:
parser.parse(r"not a or b and c")

[Call(Symbol('.or'), [Call(Symbol('.not'), [Symbol('a')]), Call(Symbol('.and'), [Symbol('b'), Symbol('c')])])]

Three comment syntaxes are supported.

In [5]:
parser.parse(r"""
one     # this is a comment
two     // also a comment
three   /* multi-line
           comment */
""")

[Symbol('one'), Symbol('two'), Symbol('three')]

Without `?`, missing names raise `QueryErrors` (as they should). With `?`, they are passed through as more missing data.

In [6]:
parser.parse(r"?safe?.navigation?.for?.possibly?.missing?.data")

[GetAttr(GetAttr(GetAttr(GetAttr(GetAttr(Symbol('safe', maybe=True), 'navigation', maybe=True), 'for', maybe=True), 'possibly', maybe=True), 'missing', maybe=True), 'data', maybe=True)]

The `if` expression is implemented as a function with a return value.

In [7]:
parser.parse(r"x = if a > 0 then b else c")

[Assignment('x', Call(Symbol('.if'), [Call(Symbol('>'), [Symbol('a'), Literal(0)]), Symbol('b'), Symbol('c')]))]

In [8]:
parser.parse(r"x = if a > 0 then b")

[Assignment('x', Call(Symbol('.if'), [Call(Symbol('>'), [Symbol('a'), Literal(0)]), Symbol('b')]))]

In [9]:
parser.parse(r"x = if has possibly_mising then a else b")

[Assignment('x', Call(Symbol('.if'), [Has(['possibly_mising']), Symbol('a'), Symbol('b')]))]

Curly brackets can be used to limit the scope of temporary variables.

In [10]:
parser.parse(r"""
result = {
    tmp1 = x + y
    tmp2 = y * z
    tmp1 / tmp2
}
""")

[Assignment('result', Block([Assignment('tmp1', Call(Symbol('+'), [Symbol('x'), Symbol('y')])), Assignment('tmp2', Call(Symbol('*'), [Symbol('y'), Symbol('z')])), Call(Symbol('/'), [Symbol('tmp1'), Symbol('tmp2')])]))]

In [11]:
parser.parse(r"""
result = { tmp1 = x + y; tmp2 = y * z; tmp1 / tmp2 }
""")

[Assignment('result', Block([Assignment('tmp1', Call(Symbol('+'), [Symbol('x'), Symbol('y')])), Assignment('tmp2', Call(Symbol('*'), [Symbol('y'), Symbol('z')])), Call(Symbol('/'), [Symbol('tmp1'), Symbol('tmp2')])]))]

### Table (set of records) expressions

Join operations apply to sets of records (like SQL's tables) and they have an operator precedence: `join` and `cross` (like `*`) bind more tightly than `union` and `except` (like `+`).

In [12]:
parser.parse(r"a union b join c cross d except e")

[Call(Symbol('.except'), [Call(Symbol('.union'), [Symbol('a'), Call(Symbol('.cross'), [Call(Symbol('.join'), [Symbol('b'), Symbol('c')]), Symbol('d')])]), Symbol('e')])]

The `as` operator turns a set into a set of records and can also be used for combinatorics (depending on whether it's *N choose 1* or *N choose k* for 1 or *k* variable names).

In [13]:
parser.parse(r"table as x")

[Pack(Symbol('table'), ['x'])]

In [14]:
parser.parse(r"table as (x, y, z)")

[Pack(Symbol('table'), ['x', 'y', 'z'])]

The `with` operator adds derived fields to a set of records. Names in the `with` expression are record fields.

In [15]:
parser.parse(r"table with { z = x + y }")

[With(Symbol('table'), [Assignment('z', Call(Symbol('+'), [Symbol('x'), Symbol('y')]))], False)]

The `where` operator selects from a set of records. Names in the `where` expression are record fields.

In [16]:
parser.parse(r"table where x > y")

[Call(Symbol('.where'), [Symbol('table'), Call(Symbol('>'), [Symbol('x'), Symbol('y')])])]

In [17]:
parser.parse(r"table with { z = x + y } where z > 10")

[Call(Symbol('.where'), [With(Symbol('table'), [Assignment('z', Call(Symbol('+'), [Symbol('x'), Symbol('y')]))], False), Call(Symbol('>'), [Symbol('z'), Literal(10)])])]

The `group by` operator restructures data inside a set of records to make a set of sets of records. Names in the `group by` expression are record fields.

In [18]:
parser.parse(r"table group by x")

[Call(Symbol('.group'), [Symbol('table'), Symbol('x')])]

The `min by` and `max by` operators replace a set of records with a single record or missing data.

In [19]:
parser.parse(r"table max by x")

[Call(Symbol('.max'), [Symbol('table'), Symbol('x')])]

### Cut and vary blocks

The `cut` and `vary` statements are the most domain-specific aspects of PartiQL, apart from histograms (below).

In [20]:
parser.parse(r"""
cut x > 0 {
    cut y > 0 {
        vary by { z = 0.01 } by { z = -0.01 } {
            x + y + z
        }
    }
}
""")

[Cut(Call(Symbol('>'), [Symbol('x'), Literal(0)]), None, None, None, [Cut(Call(Symbol('>'), [Symbol('y'), Literal(0)]), None, None, None, [Vary([Trial([Assignment('z', Literal(0.01))], None), Trial([Assignment('z', Call(Symbol('*-1'), [Literal(0.01)]))], None)], [Call(Symbol('+'), [Call(Symbol('+'), [Symbol('x'), Symbol('y')]), Symbol('z')])])])])]

The `cut` statement can be modified by a `weight`, `name` (for lookup), and `title` (for human-readability), in any order.

In [21]:
parser.parse(r"""
cut x > 0 weight by w named "cut1" titled "first cut" {
    x + y
}
""")

[Cut(Call(Symbol('>'), [Symbol('x'), Literal(0)]), Symbol('w'), Literal('cut1'), Literal('first cut'), [Call(Symbol('+'), [Symbol('x'), Symbol('y')])])]

A `vary` statement takes arbitrarily many sets of assignments, each of which can be named.

In [22]:
parser.parse(r"""
vary
  by {
      x = 0.1
      y = 0
      z = 0
  } named "x"
  by {
      x = 0
      y = 0.001
      z = 0
  } named "y"
  by {
      x = 0
      y = 0
      z = 0.01
  } named "z"
  {
      x + y + z
  }
""")

[Vary([Trial([Assignment('x', Literal(0.1)), Assignment('y', Literal(0)), Assignment('z', Literal(0))], Literal('x')), Trial([Assignment('x', Literal(0)), Assignment('y', Literal(0.001)), Assignment('z', Literal(0))], Literal('y')), Trial([Assignment('x', Literal(0)), Assignment('y', Literal(0)), Assignment('z', Literal(0.01))], Literal('z'))], [Call(Symbol('+'), [Call(Symbol('+'), [Symbol('x'), Symbol('y')]), Symbol('z')])])]

In [23]:
parser.parse(r"""
vary by { x = 0.1; y = 0;     z = 0    } named "x"
     by { x = 0;   y = 0.001; z = 0    } named "y"
     by { x = 0;   y = 0;     z = 0.01 } named "z"
  {
      x + y + z
  }
""")

[Vary([Trial([Assignment('x', Literal(0.1)), Assignment('y', Literal(0)), Assignment('z', Literal(0))], Literal('x')), Trial([Assignment('x', Literal(0)), Assignment('y', Literal(0.001)), Assignment('z', Literal(0))], Literal('y')), Trial([Assignment('x', Literal(0)), Assignment('y', Literal(0)), Assignment('z', Literal(0.01))], Literal('z'))], [Call(Symbol('+'), [Call(Symbol('+'), [Symbol('x'), Symbol('y')]), Symbol('z')])])]

### Histograms

A histogram (`hist` statement) may be as simple as a single variable.

In [24]:
parser.parse(r"hist x")

[Histogram([Axis(Symbol('x'), None)], None, None, None)]

It may be a multidimensional histogram over several variables.

In [25]:
parser.parse(r"hist x, y, z")

[Histogram([Axis(Symbol('x'), None), Axis(Symbol('y'), None), Axis(Symbol('z'), None)], None, None, None)]

And each of these variables may be given an explicit binning.

In [26]:
parser.parse(r"""
hist x by regular(100, -5, 5),
     y by regular(10, 0, 0.01)
""")

[Histogram([Axis(Symbol('x'), Call(Symbol('regular'), [Literal(100), Call(Symbol('*-1'), [Literal(5)]), Literal(5)])), Axis(Symbol('y'), Call(Symbol('regular'), [Literal(10), Literal(0), Literal(0.01)]))], None, None, None)]

The histogram (as a whole, not each axis) may have a `weight`, `name` (for lookup), and `title` (for human-readability), in any order.

In [27]:
parser.parse(r"""
hist x weight by w named "h" titled "my first histogram"
""")

[Histogram([Axis(Symbol('x'), None)], Symbol('w'), Literal('h'), Literal('my first histogram'))]

Histograms may be placed inside `cut` and `vary` statements, temporary variable blocks, or a set of records' `with` expression. In any case, it will be included in the hierarchy of counters. The number of times it is filled depends on *where* it is placed—if it is in a set of records' `with` expression, it will have an entry for each of those records (e.g. for each particle).

In [28]:
parser.parse(r"""
hist x titled "One for every event."

cut x < 0 {
    hist x titled "One for every event in which x < 0."

    table with {
        hist x titled "One for every element of table for every event in which x < 0."
    }
}
""")

[Histogram([Axis(Symbol('x'), None)], None, None, Literal('One for every event.')),
 Cut(Call(Symbol('<'), [Symbol('x'), Literal(0)]), None, None, None, [Histogram([Axis(Symbol('x'), None)], None, None, Literal('One for every event in which x < 0.')), With(Symbol('table'), [Histogram([Axis(Symbol('x'), None)], None, None, Literal('One for every element of table for every event in which x < 0.'))], False)])]

### Macros

Macros look like functions, but they can be used anywhere (as statements, expressions, etc.) to reduce repetition in the source code. They act during parsing, so they are literally syntactic sugar—not runtime objects. Macros can call other macros, but not recursively.

In [29]:
parser.parse(r"""
def add1(x) {
    x + 1
}

z + add1(y)
""")

[Call(Symbol('+'), [Symbol('z'), Call(Symbol('+'), [Symbol('y'), Literal(1)])])]

In [30]:
parser.parse(r"""
def goodbinning(x, name) {
    hist x by regular(100, 0, 50) named name
}

goodbinning(a, "a")
goodbinning(b, "b")
goodbinning(c, "c")
""")

[Histogram([Axis(Symbol('a'), Call(Symbol('regular'), [Literal(100), Literal(0), Literal(50)]))], None, Literal('a'), None),
 Histogram([Axis(Symbol('b'), Call(Symbol('regular'), [Literal(100), Literal(0), Literal(50)]))], None, Literal('b'), None),
 Histogram([Axis(Symbol('c'), Call(Symbol('regular'), [Literal(100), Literal(0), Literal(50)]))], None, Literal('c'), None)]